# QCoDeS Example with Tektronix AWG70002A

In [1]:
from qcodes.instrument_drivers.tektronix.AWG70002A import AWG70002A

In [2]:
awg = AWG70002A('awg', 'TCPIP0::172.20.2.101::inst0::INSTR')

Connected to: TEKTRONIX AWG70002A (serial:B020397, firmware:FV:5.3.0128.0) in 0.07s


In [3]:
awg.IDN()

{'firmware': 'FV:5.3.0128.0',
 'model': 'AWG70002A',
 'serial': 'B020397',
 'vendor': 'TEKTRONIX'}